In [1]:
!pip install openpyxl
!pip install requests
!pip install pandas-gbq --user

Inputs

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'datalake-analytics-339922'

packages

In [3]:
import requests
import os
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
print("packages is: ok")

packages is: ok


clients

In [4]:
bq = bigquery.Client(project = PROJECT_ID)
print("client is: ok")

client is: ok


In [35]:
query = f"""
SELECT *
FROM `datalake-analytics-339922.UniDataSentiment.sss_onsite_gpt`
"""
#use the on-site
#datalake-analytics-339922.UniDataSentiment.sss_onsite_gpt
df_bq_all = bq.query(query = query).to_dataframe()
print("df_bq_all is: ok")
len(df_bq_all)
print(df_bq_all.shape)
#df_bq_all.to_excel('data/data_on_sentiments_28_colums.xlsx',index=False)
#print('data is ready')

df_bq_all is: ok
(1365, 28)


get names of colums of dataframe of bigquery

In [31]:
colum_list = df_bq_all.columns.tolist()
len(colum_list)

28

MERGE ORIGINAL

In [32]:
new_df_to_bq = pd.read_excel('data/data  1-5 feb on - site.xlsx')
print(new_df_to_bq.shape)
new_df_to_bq.columns = colum_list
new_df_to_bq

(117, 28)


,Id_Model,User,Message_x_x_x_x_x_x_x,Brands,Subsidiary,Sentiment_H,Topics_1_x,Topics_2_x,Topics_3_x,Locations,...,First_Admin_Comment_x,Link,Sentiment_x,Clasification_M_x,Clasify_IA_1,Clasify_IA_2,Clasify_IA_3,Clasify_IA_4,Clasify_IA_5,Clasify_IA_6
0,"model23020711233200000000002escv5f""",Manuelo León,Que nos permitan hacer cambios de medico prima...,Triple-S Advantage,ADVANTAGE,Negative,Planes y servicios,Servicio al cliente,NaN,NaN,...,NaN,https://www.facebook.com/581325368651341/posts...,0,Planes y servicios,Rabia,Ninguno,"gestiones, Internet, cambios",Quejas,Ninguno,Ninguno
1,"model23020711233200000000003escv5f""",Cira Figueroa,Gracias a Triple-S por la oportunidad de parti...,Triple-S Advantage,ADVANTAGE,Positive,Institucional,Responsabilidad social,NaN,NaN,...,NaN,https://www.facebook.com/581325368651341/posts...,2,Planes y servicios,Alegría,Ninguno,-Triple-S-Damaris Pérez-Nitza Pérez,Educación,Satisfacción,Ninguno
2,"model23020711233200000000004escv5f""",Iris Rosado,Como puedo saber el balance de mis otc,Triple-S Advantage,ADVANTAGE,Neutral,Planes y servicios,Medicamentos OTC,Balances,NaN,...,NaN,https://www.facebook.com/581325368651341/posts...,1,Planes y servicios,Ninguno,Ninguno,There are a number of ways brands can use digi...,Finanzas,Ninguno,Ninguno
3,"model23020711233200000000005escv5f""",José Morale,La prediabetes es reversible?,Triple-S Advantage,ADVANTAGE,Neutral,Institucional,Responsabilidad social,NaN,NaN,...,NaN,https://www.facebook.com/581325368651341/posts...,1,Comunicación oficial,Ninguno,Ninguno,"reversible, prediabetes, importancia",Salud,Curiosidad,Ninguno
4,"model23020711233200000000006escv5f""",Fernando Matias,"Esa artista, un abrazo 🤗",Triple-S Advantage,ADVANTAGE,Positive,Institucional,Responsabilidad social,NaN,NaN,...,NaN,https://www.facebook.com/581325368651341/posts...,2,Institucional,Alegría,Ninguno,"Artista, abrazo, esa.",Cultura,Alegría,Ninguno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,"model23020711233200000000114escv5f""",Roberto Rivera,El.mani.te.puede.producir\nChuras.y.o..estreni...,Clínicas Salus,SALUS,Neutral,Comunicación oficial,Educativos,NaN,NaN,...,NaN,https://www.facebook.com/1025905351405345/post...,2,Institucional,Rabia,Ninguno,"churas, estrenimiento, maní",Salud,Ninguno,Ninguno
113,"model23020711233200000000115escv5f""",Alejandro Pagan,"Ana Fuentes No, se pero yo consumo mucho la m...",Clínicas Salus,SALUS,Neutral,Comunicación oficial,Educativos,NaN,NaN,...,NaN,https://www.facebook.com/1025905351405345/post...,1,Planes y servicios,Alegría,Neutro,", peanut, butter.Creamy, peanut, butter",Cobertura,Satisfacción,Coches y Vehículos
114,"model23020711233200000000116escv5f""",Maria Oliver,Pero es muy saludable si lo ligas con jalea d...,Clínicas Salus,SALUS,Neutral,Comunicación oficial,Educativos,NaN,NaN,...,NaN,https://www.facebook.com/1025905351405345/post...,0,Comunicación oficial,Ninguno,Neutro,"saludable, jalea, uva",Salud,Satisfacción,Coches y Vehículos
115,"model23020711233200000000117escv5f""",Ramona Baez,"El maní es bueno y saludable, para los que ...",Clínicas Salus,SALUS,Neutral,Comunicación oficial,Educativos,NaN,NaN,...,NaN,https://www.facebook.com/1025905351405345/post...,0,Planes y servicios,Ninguno,Ninguno,"Maní, Alergia, Saludable",Salud,Ninguno,Ninguno


In [33]:
table_name = 'datalake-analytics-339922.UniDataSentiment.sss_onsite_gpt'
print("source data : ok")


source data : ok


Create new table on bigquery Note: if bigquery already exist don't use this part of code justo upload the data

We create a schema

In [ ]:
# Crea una lista vacía para almacenar los nombres de las columnas que ya se han procesado
columnas = [str(col) for col in new_df_to_bq.columns]
columnas_tupla = [(col, "STRING", "NULLABLE") for col in columnas]
schema = [bigquery.SchemaField(*col_tuple) for col_tuple in columnas_tupla]
#for i in :
    #columnas[i] =  columnas[i].apply(lambda x: x + '2')
len(schema)
#bigquery.SchemaField("col2", "STRING", mode="NULLABLE"),

22

the next code is to create code

In [68]:
# Nombre de la tabla en BigQuery


# Crea una instancia de "bigquery.Client"
client = bigquery.Client()

# Crea una instancia de "bigquery.Table" con el esquema y la tabla especificados
table = bigquery.Table(table_name, schema=schema)

# Crea la tabla en BigQuery
table = client.create_table(table)  # API request
print(f"Created table {table.table_id}")
#datalake-analytics-339922.UniDataSentiment.sss_offsite_unidata_final



Created table sss_onsite_unidata_openIA_V4


upload the data  on bigquery  Nota : Use the method "append" to joint info without replace

In [34]:
# Convierte todas las columnas del DataFrame a tipo "string"
lastdf =new_df_to_bq.astype(str)
lastdf.to_gbq(table_name, if_exists='append')
print ("dataframe is : ok")


100%|██████████| 1/1 [00:00<00:00, 6990.51it/s]

dataframe is : ok
